In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [7]:
ml_data =  pd.read_csv('../data/chrome_data.csv')
ml_data.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes,no,SAID_YES
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,Chrome Mobile,1,0,1
1,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,Chrome Mobile,0,1,0


In [23]:
columns_to_be_dropped = ['no','SAID_YES']
ml_data_clean = ml_data.drop(columns_to_be_dropped,axis=1)
ml_data_clean.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,Chrome Mobile,1
1,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,Chrome Mobile,0


In [25]:
catagorical_columns = ml_data_clean.select_dtypes(include='object').columns.tolist()
numerical_columns = ml_data_clean.select_dtypes(exclude='object').columns.tolist()

print("Numerical Columns:", numerical_columns)
print("****************")
print("Categorical Columns:", catagorical_columns)

Numerical Columns: ['hour', 'yes']
****************
Categorical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']


In [26]:
ml_data_clean[catagorical_columns].describe()

,auction_id,experiment,date,device_make,browser
count,695,695,695,695,695
unique,695,2,8,14,1
top,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-03,Generic Smartphone,Chrome Mobile
freq,1,371,176,665,695


In [36]:
catagorical_columns
for column in catagorical_columns:
  le = LabelEncoder()
  ml_data_clean[column] = le.fit_transform(ml_data_clean[column])
ml_data_clean.head(5)

,auction_id,experiment,date,hour,device_make,browser,yes
0,0,1,1,16,1,0,1
1,1,1,3,8,1,0,0
2,2,0,0,15,1,0,0
3,3,1,7,2,1,0,0
4,4,0,0,15,1,0,1
